In [ ]:
%matplotlib widget
import flammkuchen as fl
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm

import lotr.plotting as pltltr
from lotr import LotrExperiment, dataset_folders
from lotr.utils import convolve_with_tau, interpolate, zscore

COLS = pltltr.COLS
from lotr.utils import pearson_regressors


def _process_reg(reg, tau_s=3):
    reg = convolve_with_tau(reg, int(tau_s * exp.fs))
    return zscore(reg)

In [ ]:
fish_with_eyes = [
    f for f in dataset_folders if "eyes" in f.name and "noeyes" not in f.name
]

In [ ]:
res_df = []

for path in tqdm(fish_with_eyes):
    exp = LotrExperiment(path)
    dlc_df = fl.load(path / "behavior_from_dlc.h5", "/data")

    fictive_head = exp.fictive_heading
    phase = exp.network_phase

    eyes_arr = dlc_df["rt_eye_medfilt"] + dlc_df["lf_eye_medfilt"]
    interp_eye = _process_reg(interpolate(dlc_df["t"], eyes_arr, exp.time_arr))

    eye_r_scores = pearson_regressors(exp.traces, interp_eye[:, np.newaxis])[0, :]
    eye_vel = np.zeros(len(interp_eye))
    eye_vel[1:] = np.diff(interp_eye)
    eye_vel_scores = pearson_regressors(exp.traces, eye_vel[:, np.newaxis])[0, :]

    hdn = np.full(len(eye_r_scores), False)
    hdn[exp.hdn_indexes] = True

    res_df.append(
        pd.DataFrame(
            dict(
                fid=path.name,
                hdn=hdn,
                scores=eye_r_scores,
                scores_vel=eye_vel_scores,
                left=exp.motor_regressors["left_1"],
                right=exp.motor_regressors["right_1"],
                any_motor=np.max(
                    [exp.motor_regressors["right_1"], exp.motor_regressors["right_1"]],
                    axis=0,
                ),
                ax0=exp.morphed_coords_um[:, 0],
                ax1=exp.morphed_coords_um[:, 1],
                ax2=exp.morphed_coords_um[:, 2],
            )
        )
    )

res_df = pd.concat(res_df).reset_index()

In [ ]:
f, ax = plt.subplots(figsize=(3, 2), gridspec_kw=dict(left=0.2, bottom=0.2))
bins = np.arange(0, 1, 0.05)
plt.hist(np.abs(res_df["scores"]), bins, density=False, fc=".6", label="All ROIs")
plt.hist(
    np.abs(res_df.loc[res_df["hdn"], "scores"]),
    bins,
    density=False,
    fc=COLS["ring"],
    label="r1π ROIs",
)
ax.set(yscale="log", ylabel="Corr. with gaze pos.", xlabel="Count")
ax.legend()
plt.show()
pltltr.despine(ax)
pltltr.savefig("histogram_eyecorr", folder="S9")

In [ ]:
f, ax = plt.subplots(figsize=(2.5, 2.5))
s = 8
ax.scatter(
    res_df["ax1"],
    res_df["ax2"],
    lw=0,
    s=s,
    alpha=0.1,
    color=(0.3,) * 3,
)

l = 0.8
s = ax.scatter(
    res_df.loc[np.abs(res_df["scores"]) > 0.25, "ax1"],
    res_df.loc[np.abs(res_df["scores"]) > 0.25, "ax2"],
    lw=0,
    s=s,
    c=res_df.loc[np.abs(res_df["scores"]) > 0.25, "scores"],
    vmin=-l,
    vmax=l,
    cmap="RdBu_r",
)

loc = [-110, -138]
ax.axis("equal")
pltltr.add_anatomy_scalebar(ax, pos=loc, length=40, fontsize=6)
pltltr.add_cbar(
    s,
    ax,
    (0.85, 0, 0.2, 0.05),
    ticks=(-l, l),
    ticklabels=("-0.8 (lf.)", "0.8 (rt.)"),
    orientation="horizontal",
    title="Corr. with gaze pos.",
    titlesize=7,
    labelsize=6,
)
pltltr.savefig("colormap_eyeregr", folder="S9")

In [ ]:
f, ax = plt.subplots(figsize=(2.5, 2.5))
s = 8
ax.scatter(
    res_df["ax1"],
    res_df["ax2"],
    lw=0,
    s=s,
    alpha=0.1,
    color=(0.3,) * 3,
)

l = 0.3
s = ax.scatter(
    res_df.loc[np.abs(res_df["scores"]) > 0.25, "ax1"],
    res_df.loc[np.abs(res_df["scores"]) > 0.25, "ax2"],
    lw=0,
    s=s,
    c=res_df.loc[np.abs(res_df["scores"]) > 0.25, "scores_vel"],
    vmin=-l,
    vmax=l,
    cmap="RdBu_r",
)

loc = [-110, -138]
ax.axis("equal")
pltltr.add_anatomy_scalebar(ax, pos=loc, length=40, fontsize=6)
pltltr.add_cbar(
    s,
    ax,
    (0.85, 0, 0.2, 0.05),
    ticks=(-l, l),
    ticklabels=("-0.8 (lf.)", "0.8 (rt.)"),
    orientation="horizontal",
    title="Corr. with eyes vel.",
    titlesize=7,
    labelsize=6,
)
pltltr.savefig("colormap_eyeregr_vel", folder="S9")